In [1]:
import pandas as pd

In [2]:
# model output

df_prediction = pd.read_csv('predictions/prediction_nmf.csv')

In [3]:
# all movie ids vs titles

df_movies = pd.read_csv('ml-latest-small/movies.csv', usecols=[0,1], names=['movieId', 'title'])

all_movies_dict = pd.Series(df_movies['title'].values,index=df_movies['movieId']).to_dict()

In [4]:
df_prediction.head()

,userId,movieId,predicted_rating,rating
0,1,157,3.201665,5.0
1,1,362,3.930156,5.0
2,1,457,4.612365,5.0
3,1,543,4.011487,4.0
4,1,590,4.099457,4.0


In [5]:
# all users in the validation set 

all_users = df_prediction['userId'].unique().tolist()

In [6]:
# recommendations as per "predicted ratings" in the validation set 

df_prediction1 = df_prediction[['userId', 'movieId', 'predicted_rating']]
df_prediction1['predicted_movies'] = df_prediction1 \
  .apply(lambda x: (x['movieId'], x['predicted_rating']), axis=1)

df_prediction2 = df_prediction1[['userId', 'predicted_movies']]

df_prediction_formatted = df_prediction2 \
  .groupby('userId')['predicted_movies'].apply(list).reset_index(name='recommendation')

df_prediction_sorted = df_prediction_formatted

df_prediction_sorted['recommendation'] = \
  df_prediction_sorted['recommendation'].apply( \
    lambda x: sorted(x, key=lambda tup: tup[1], reverse=True))

df_prediction_sorted.head()

sorted_reco_by_userid = pd.Series( \
  df_prediction_sorted['recommendation'].values,index=df_prediction_sorted['userId']).to_dict()

In [7]:
# this function returns top N recommendations for a given user
# based on the predicted ratings, N = 10 by default

def get_top_n_recommendations(user, N=10):
  top_n_reco = sorted_reco_by_userid[user][:N]
  
  return [all_movies_dict[str(int(x[0]))] for x in top_n_reco]

In [8]:
# recommendations as per "true ratings" in the validation set

df_true1 = df_prediction[['userId', 'movieId', 'rating']]
df_true1['movies'] = df_true1 \
  .apply(lambda x: (x['movieId'], x['rating']), axis=1)

df_true2 = df_true1[['userId', 'movies']]

df_true_formatted = df_true2 \
  .groupby('userId')['movies'].apply(list).reset_index(name='recommendation')

df_true_sorted = df_true_formatted

df_true_sorted['recommendation'] = \
  df_true_sorted['recommendation'].apply( \
    lambda x: sorted(x, key=lambda tup: tup[1], reverse=True))

df_true_sorted.head()

sorted_true_ratings_by_userid = pd.Series( \
  df_true_sorted['recommendation'].values,index=df_true_sorted['userId']).to_dict()

In [9]:
# this function returns top N recommendations for a given user
# based on the true ratings, N = 10 by default

def get_top_n_true_ratings(user, N=10):
  top_n_true_ratings = sorted_true_ratings_by_userid[user][:N]
  
  return [all_movies_dict[str(int(x[0]))] for x in top_n_true_ratings]

In [10]:
def calculate_metrics(user, threshold = 1.5):
    true_ratings = sorted_true_ratings_by_userid[user]
    predicted_ratings = sorted_reco_by_userid[user]
    true_rating_dict = dict(true_ratings)
    predicted_rating_dict = dict(predicted_ratings)
    false_positives = 0
    false_negatives = 0
    true_positives = 0
    for movie, true_rating in true_rating_dict.items():
#         print(movie, true_rating, predicted_rating_dict[movie])
        diff = abs(true_rating - predicted_rating_dict[movie])
        if diff > threshold:
            if true_rating < predicted_rating_dict[movie]:
                false_positives += 1
            else:
                false_negatives += 1
        else:
            true_positives += 1
    precision = 0
    if true_positives + false_positives != 0:
        precision = true_positives / (true_positives + false_positives)
    recall = 0
    if true_positives + false_negatives != 0:
        recall = true_positives / (true_positives + false_negatives)
    return [precision, recall]

In [11]:
# get recommendation for each user in the validation set

top_n_recommendations_by_user = {}

user_out = []
for user in all_users:
    out = []

    # user
    out.append(user)

    # top N recommendations (n = 10 by default)
    top_n_reco_for_user = get_top_n_recommendations(user)
    top_n_recommendations_by_user[user] = top_n_reco_for_user

#     top_n_true_ratings_for_user = get_top_n_true_ratings(user)
    
    
    out.extend(calculate_metrics(user))


    user_out.append(out)
#     print(set(top_n_reco_for_user) == set(top_n_true_ratings_for_user), out)

In [12]:
# Recommendation evaluation for all users in the validation set

df_out = pd.DataFrame(user_out, columns=['userId', 'precision_user', 'recall_user'])
df_out

,userId,precision_user,recall_user
0,1,0.977778,0.977778
1,2,1.000000,1.000000
2,3,0.600000,0.500000
3,4,0.800000,0.914286
4,5,1.000000,1.000000
...,...,...,...
605,606,0.986364,0.986364
606,607,0.941176,0.914286
607,608,0.949367,0.949367
608,609,1.000000,1.000000


In [13]:
# top N recommendations for a user example

top_n_recommendations_by_user[7]

['Rocky (1976)',
 'Sideways (2004)',
 'As Good as It Gets (1997)',
 'Beautiful Mind, A (2001)',
 'Ray (2004)',
 'Twilight Samurai, The (Tasogare Seibei) (2002)',
 'Austin Powers: The Spy Who Shagged Me (1999)',
 'Jurassic Park (1993)',
 'Lord of the Rings: The Two Towers, The (2002)',
 'X-Men (2000)']

In [14]:
# EVALUATION

# mae
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(df_prediction['rating'], df_prediction['predicted_rating'])
print('mae', mae)

# rmse
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(df_prediction['rating'], df_prediction['predicted_rating']))
print('rmse', rmse)

# precision
precision = df_out['precision_user'].mean()
print('avg precision', precision)

# recall
recall = df_out['recall_user'].mean()
print('avg recall', recall)

# f measure
f_measure = (2.0 * precision * recall) / (precision + recall)
print('f measure', f_measure)

mae 0.7101148582937473
rmse 0.9278849573626533
avg precision 0.922696573537139
avg recall 0.9444908277670366
f measure 0.9334665067996909
